In [ ]:
import inro.modeller as _m
import inro.emme.desktop as _d
import csv
import os
import multiprocessing
import numpy as np
import pandas as pd
import sqlite3
import traceback as _traceback
import datetime

import shutil

In [ ]:
dt = _d.app.connect()
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()

# Define a list of name and path

In [ ]:
# shape file used to do tag_polytolink (example: RTM_8ZonePolygon.shp), use None to skip this step
shp_file = None
# text file with link attribute by polygon zone (example: @toll by @msczone), use None to skip this step
custom_network_txt = None

params = {
    # test scenarios #
    'run_test_1': {
        'master_scen_name': "test_1",
        'master_scen': 1000,
        'horizon_year': 2011,
        'global_iterations': 2, #4,
        'max_distribution_iterations': 2, #60,
        'max_assignment_iterations': 2, #200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': False,
        'scenario_name': 'test',
        'alternative_name': 'test',
        'demographic_file_override': None #'taz1700_demographics_2050-20.csv'
    },
    'run_test_2': {
        'master_scen_name': "test_2",
        'master_scen': 2000,
        'horizon_year': 2016,
        'global_iterations': 2, #4,
        'max_distribution_iterations': 2, #60,
        'max_assignment_iterations': 2, #200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': False,
        'scenario_name': 'test',
        'alternative_name': 'test',
        'demographic_file_override': None #'taz1700_demographics_2050-20.csv'
    },
    'run_test_3': {
        'master_scen_name': "test_3",
        'master_scen': 3000,
        'horizon_year': 2035,
        'global_iterations': 2, #4,
        'max_distribution_iterations': 2, #60,
        'max_assignment_iterations': 2, #200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': False,
        'scenario_name': 'test',
        'alternative_name': 'test',
        'demographic_file_override': None #'taz1700_demographics_2050-20.csv'
    },
    'run_test_4': {
        'master_scen_name': "test_4",
        'master_scen': 4000,
        'horizon_year': 2050,
        'global_iterations': 2, #4,
        'max_distribution_iterations': 2, #60,
        'max_assignment_iterations': 2, #200,
        'run_congested_transit': True,
        'run_capacited_transit': True,
        'num_processors': multiprocessing.cpu_count(),
        'run_parking_model': True,
        'run_toll_skim': False,
        'scenario_name': 'test',
        'alternative_name': 'test',
        'demographic_file_override': None #'taz1700_demographics_2050-20.csv'
    }
}


# Loop through and initialize all emme databanks

In [ ]:
initEmmebank = _m.Modeller().tool("translink.RTM3.init_emmebank")

In [ ]:
for databank_name in params.keys():
    try:
        print "{} initiating databank ...".format(databank_name)

        initEmmebank(
            folder_name="db_{}".format(databank_name),
            title_name=databank_name,
            master_scen=params[databank_name]['master_scen'],
            master_scen_name=params[databank_name]['master_scen_name'])

        print "{} successfully created.".format(databank_name)

    except Exception as e:
        print("Databank creation for {} failed.".format(databank_name), e)

# Loop through and run all the ebs open in desktop (except minimal base) 

## Get List of Scenarios from Desktop

In [ ]:
# reconnect to refresh session

dt = _d.app.connect()
de = dt.data_explorer()
db = de.active_database()
ebs = de.databases()

In [ ]:
# make sure modeller is closed or it will print to the python console in there
print "===================================="
for eb in ebs:
    print eb.title()
    for scen in list(eb.scenarios()):
        print "{}\t{}".format(scen.number(), scen.title())
    print "===================================="

In [ ]:
run_model = _m.Modeller().tool("translink.RTM3.runmodel")
util = _m.Modeller().tool("translink.util")

if shp_file is not None:
    tag_polytolink = _m.Modeller().tool("translink.internal_tools.tagpolygon")

## Run Models

In [ ]:
for eb in ebs:
    title = eb.title()

    if title == 'Minimal Base Databank':
        continue

    if title not in params.keys():
        continue

    eb.open()
    
    current_time = datetime.datetime.now()
    print "\nstarting {} run at {}".format(title, current_time)
    
    try:
        year = params[title]['horizon_year']
        ip = util.get_input_path(eb)
        geoFile = os.path.join(ip, 'taz1700_geographics_{}.csv'.format(year))

        if params[title]['demographic_file_override'] is None:
            demoFile = os.path.join(ip, 'taz1700_demographics_{}.csv'.format(year))
        else:
            demoFile = os.path.join(ip, params[title]['demographic_file_override'])
            print "{} custom demographics file: {}.".format(title, params[title]['demographic_file_override'])

        scen = eb.scenario_by_number(
            params[title]['master_scen']).core_scenario
        
        # copy custom network txt file - please define custom_network_txt
        if custom_network_txt is not None:
            shutil.copyfile(custom_network_txt, os.path.join(ip, 'custom_network.txt'))

        if shp_file is not None:
            print "{} running polygon to link tagging ...".format(title)
            # run polygon link tagging - please define shp_file
            tag_polytolink(
                scenario=scen,
                linkattributeName="@msczone",
                polygonfile=shp_file,
                polygonfield="Id",
                algorithm="by_length",
                excludeconnector=True,
                default = 0)

        print "{} running model ...".format(title)

        run_model(
            horizon_year=year,
            global_iterations=params[title]['global_iterations'],
            master_scen=scen,
            demographics_file=demoFile,
            geographics_file=geoFile,
            max_distribution_iterations=params[title]
            ['max_distribution_iterations'],
            max_assignment_iterations=params[title]
            ['max_assignment_iterations'],
            run_congested_transit=params[title]['run_congested_transit'],
            run_capacited_transit=params[title]['run_capacited_transit'],
            num_processors=params[title]['num_processors'],
            run_parking_model=params[title]['run_parking_model'],
            run_toll_skim=params[title]['run_toll_skim'],
            scenario_name=params[title]['scenario_name'],
            alternative_name=params[title]['alternative_name'])

        current_time = datetime.datetime.now()
        print "{} ran successfully at {}".format(title, current_time)

    except Exception as e:
        print("Scenario {} run failed.".format(title), e)